In [7]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langgraph_supervisor import create_supervisor
import langchain

model = ChatOllama(
    model="qwen2.5:7b",           
    temperature=0,            
)

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    system_prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    system_prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    system_prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what's the combined headcount of the FAANG companies in 2024?
================================== Ai Message ==================================
Name: supervisor

I don't have real-time data access to provide you with the exact combined headcount for the FAANG companies (Facebook, Amazon, Apple, Netflix, and Google) for the year 2024. However, I can help you understand how to find this information or suggest where you might look it up.

Would you like me to guide you on how to estimate this number based on available data from previous years or direct you to reliable sources that might have projections?


In [ ]:
# from langgraph.prebuilt import create_react_agent
# from langgraph_supervisor import create_supervisor

# #from langchain_openai import ChatOpenAI

# from langgraph.func import entrypoint, task
# from langgraph.graph import add_messages

# from langchain_ollama.llms import OllamaLLM
# from langchain_core.prompts import ChatPromptTemplate
# from vector import retriever

# model = OllamaLLM(model="qwen2.5:7b")

# # Create specialized agents

# # Functional API - Agent 1 (Joke Generator)
# @task
# def generate_joke(messages):
#     """First LLM call to generate initial joke"""
#     system_message = {
#         "role": "system", 
#         "content": "Write a short joke"
#     }
#     msg = model.invoke(
#         [system_message] + messages
#     )
#     return msg

# @entrypoint()
# def joke_agent(state):
#     joke = generate_joke(state['messages']).result()
#     messages = add_messages(state["messages"], [joke])
#     return {"messages": messages}

# joke_agent.name = "joke_agent"

# # Graph API - Agent 2 (Research Expert)
# def web_search(query: str) -> str:
#     """Search the web for information."""
#     return (
#         "Here are the headcounts for each of the FAANG companies in 2024:\n"
#         "1. **Facebook (Meta)**: 67,317 employees.\n"
#         "2. **Apple**: 164,000 employees.\n"
#         "3. **Amazon**: 1,551,000 employees.\n"
#         "4. **Netflix**: 14,000 employees.\n"
#         "5. **Google (Alphabet)**: 181,269 employees."
#     )

# research_agent = create_react_agent(
#     model=model,
#     tools=[web_search],
#     name="research_expert",
#     prompt="You are a world class researcher with access to web search. Do not do any math."
# )

# # Create supervisor workflow
# workflow = create_supervisor(
#     [research_agent, joke_agent],
#     model=model,
#     prompt=(
#         "You are a team supervisor managing a research expert and a joke expert. "
#         "For current events, use research_agent. "
#         "For any jokes, use joke_agent."
#     )
# )

# # Compile and run
# app = workflow.compile()
# result = app.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "Share a joke to relax and start vibe coding for my next project idea."
#         }
#     ]
# })

# for m in result["messages"]:
#     m.pretty_print()